In [28]:
# trd
import MetaTrader5 as mt

# processing
import pandas as pd
from datetime import datetime

---

In [29]:
mt.initialize()

True

In [30]:
login = 0
password = '' 
server = ''

mt.login(login, password, server)

True

In [31]:
mt.account_info()

AccountInfo(login=152702909, trade_mode=0, leverage=2, limit_orders=1024, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=527.04, credit=0.0, profit=0.0, equity=527.04, margin=0.0, margin_free=527.04, margin_level=0.0, margin_so_call=30.0, margin_so_so=0.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='DEMO', server='Exness-MT5Trial', currency='USD', company='Exness Technologies Ltd')

In [32]:
df = pd.DataFrame(mt.copy_rates_range('BTCUSD', mt.TIMEFRAME_D1, datetime(2023, 1, 1), datetime.now()))
df.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,1672531200,16532.88,16619.48,16492.71,16603.49,79688,427,0
1,1672617600,16602.97,16773.48,16535.88,16665.66,82223,747,0
2,1672704000,16665.36,16773.61,16593.15,16662.49,82161,575,0
3,1672790400,16661.05,16988.17,16639.02,16837.79,83272,371,0
4,1672876800,16839.31,16868.59,16750.72,16814.80,82185,517,0


In [33]:
df.time = pd.to_datetime(df.time, unit='s', utc=True)
df.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,2023-01-01 00:00:00+00:00,16532.88,16619.48,16492.71,16603.49,79688,427,0
1,2023-01-02 00:00:00+00:00,16602.97,16773.48,16535.88,16665.66,82223,747,0
2,2023-01-03 00:00:00+00:00,16665.36,16773.61,16593.15,16662.49,82161,575,0
3,2023-01-04 00:00:00+00:00,16661.05,16988.17,16639.02,16837.79,83272,371,0
4,2023-01-05 00:00:00+00:00,16839.31,16868.59,16750.72,16814.80,82185,517,0


In [34]:
df.tail()

,time,open,high,low,close,tick_volume,spread,real_volume
408,2024-02-13 00:00:00+00:00,49894.69,50323.71,48278.14,49733.91,163269,1627,0
409,2024-02-14 00:00:00+00:00,49710.36,52021.73,49239.29,51740.52,172123,1626,0
410,2024-02-15 00:00:00+00:00,51768.09,52779.51,51255.39,51908.33,172251,1650,0
411,2024-02-16 00:00:00+00:00,51886.93,52559.49,51547.60,52119.51,154540,1625,0
412,2024-02-17 00:00:00+00:00,52112.88,52177.36,51572.66,51708.30,43603,1796,0


In [35]:
mt.orders_get()

()

In [36]:
mt.positions_total()

0

In [37]:
mt.positions_get()

()

In [45]:
# prepare the buy request structure
lot = 0.01
symbol = "BTCUSD"
request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt.ORDER_TYPE_BUY,
    "price": mt.symbol_info_tick(symbol).ask,
    "sl": mt.symbol_info_tick(symbol).ask - 1000,
    "tp": mt.symbol_info_tick(symbol).ask + 1000,
    "comment": "python script buy",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_IOC,
}

# execute order
result = mt.order_send(request)
result

OrderSendResult(retcode=10009, deal=363851349, order=575977263, volume=0.01, price=51744.18, bid=51709.700000000004, ask=51744.18, comment='python script bu', request_id=2878956848, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BTCUSD', volume=0.01, price=51744.18, stoplimit=0.0, sl=50744.18, tp=52744.18, deviation=0, type=0, type_filling=1, type_time=0, expiration=0, comment='python script buy', position=0, position_by=0))

In [46]:
result.order

575977263

In [47]:
# prepare the buy request structure
request = {
    "action": mt.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt.ORDER_TYPE_SELL,
    "position": result.order,
    "price": mt.symbol_info_tick(symbol).bid,
    "comment": "python script close",
    "type_time": mt.ORDER_TIME_GTC,
    "type_filling": mt.ORDER_FILLING_IOC,
}

# execute order
result = mt.order_send(request)
result

OrderSendResult(retcode=10009, deal=363851543, order=575977435, volume=0.01, price=51746.37, bid=51746.37, ask=51775.23, comment='python script cl', request_id=2878956849, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BTCUSD', volume=0.01, price=51746.37, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=1, type_filling=1, type_time=0, expiration=0, comment='python script close', position=575977263, position_by=0))

In [48]:
mt.order_check(request)

OrderCheckResult(retcode=10013, balance=0.0, equity=0.0, profit=0.0, margin=0.0, margin_free=0.0, margin_level=0.0, comment='Invalid request', request=TradeRequest(action=1, magic=0, order=0, symbol='BTCUSD', volume=0.01, price=51746.37, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=1, type_filling=1, type_time=0, expiration=0, comment='python script close', position=575977263, position_by=0))

In [49]:
mt.shutdown()

True